In [1]:
import numpy as np
import pandas as pd
import os, sys
import matplotlib.pyplot as plt
import sys
from PIL import Image, ImageOps
from pathlib import Path
from keras.preprocessing.image import img_to_array
from sklearn import preprocessing

from keras.datasets import mnist
import numpy as np
import numpy as np
import pandas as pd
import os, sys
import matplotlib.pyplot as plt
import sys
from PIL import Image, ImageOps
from pathlib import Path

from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import keras
from keras.models import Sequential
from keras.layers import Dense, Conv2D , MaxPool2D , Flatten , Dropout 
from keras.preprocessing.image import ImageDataGenerator
#from keras.optimizers import Adam

from sklearn.metrics import classification_report,confusion_matrix
from tensorflow.keras.utils import to_categorical

import tensorflow as tf

import time
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MultipleLocator
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from sklearn import metrics
from tensorflow.keras.regularizers import l2

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions

from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten 
from keras.layers import MaxPooling2D, Dropout

from scipy.ndimage import gaussian_filter


# Data Cleaning & Processing

In [2]:
# importing and resizing image data 
# change path to wherever your data lives on your computer
csv_path = "../petfinder-pawpularity-score/train.csv"
image_path = '../petfinder-pawpularity-score/train/'
model = ResNet50(weights='imagenet')

def CutenessBuckets(pawpularity_score):
    if pawpularity_score <= 10:
        return 0
    elif pawpularity_score <= 20:
        return 1
    elif pawpularity_score <= 30:
        return 2
    elif pawpularity_score <= 40:
        return 3
    elif pawpularity_score <= 50:
        return 4
    elif pawpularity_score <= 60:
        return 5
    elif pawpularity_score <= 70:
        return 6
    elif pawpularity_score <= 80:
        return 7
    elif pawpularity_score <= 90:
        return 8
    elif pawpularity_score <= 100:
        return 9

def getAdditionalTags(model, imArr):
    tags=[]
    im = np.expand_dims(imArr, axis=0)
    im = preprocess_input(im)
    preds = model.predict(im)
    #print('Predicted:', decode_predictions(preds, top=3)[0])
    return decode_predictions(preds, top=3)[0][0][0], decode_predictions(preds, top=3)[0][1][0], decode_predictions(preds, top=3)[0][2][0] 
    #return tags

def CleanandProcessData(csv_path, image_path, image_resize):

    # creating path to image using the id in the data frame we've created
    df['path'] = image_path + df['Id'] + '.jpg'
    df['img'] = df['path'].apply(lambda x: Image.open(x).convert("RGB").resize((image_resize,image_resize)))
    print('resized')
    df['cuteness_bin'] = df['Pawpularity'].apply(CutenessBuckets)
    df['img_array'] = df['img'].apply(lambda x: img_to_array(x)/255)

    labels_raw=[]
    labels_bin = []
    imgArr=[]
    tags=[]
    print('resnet and tags processing')
    for ind in df.index:
        tags_data=[]
        img = df.img_array[0]
        #t1,t2,t3=getAdditionalTags(model, img)
        imgArr.append(img)
        labels_raw.append(df['Pawpularity'][ind])
        labels_bin.append(df['cuteness_bin'][ind])
        #append all metadata in the dataframe as tags
        tags_data.append(df['Subject Focus'][ind])
        tags_data.append(df['Eyes'][ind])
        tags_data.append(df['Face'][ind])
        tags_data.append(df['Near'][ind])
        tags_data.append(df['Accessory'][ind])
        tags_data.append(df['Group'][ind])
        tags_data.append(df['Collage'][ind])
        tags_data.append(df['Human'][ind])
        tags_data.append(df['Occlusion'][ind])
        tags_data.append(df['Info'][ind])
        tags_data.append(df['Blur'][ind])            
        tags.append(np.array(tags_data, dtype=int))
    #print(tags_data)
    labels_raw = np.array(labels_raw)
    labels_bin = np.array(labels_bin)
    tags= np.array(tags,dtype=int)

    X_train=np.array(imgArr, dtype=float)
    print('Training Shape:',X_train.shape)
    print('Labels Shape;', labels_raw.shape)
    return X_train, labels_raw, labels_bin, tags

# imnporting csv with image ids and pawpularity scores
print('2')
df = pd.read_csv(csv_path)
X_train, labels_raw, labels_bin, tags = CleanandProcessData(csv_path, image_path, 50)
print(df.head)
print(tags[0])
print (type(tags))



2
resized
resnet and tags processing
Training Shape: (9912, 50, 50, 3)
Labels Shape; (9912,)
<bound method NDFrame.head of                                     Id  Subject Focus  Eyes  Face  Near  \
0     0007de18844b0dbbb5e1f607da0606e0              0     1     1     1   
1     0009c66b9439883ba2750fb825e1d7db              0     1     1     0   
2     0013fd999caf9a3efe1352ca1b0d937e              0     1     1     1   
3     0018df346ac9c1d8413cfcc888ca8246              0     1     1     1   
4     001dc955e10590d3ca4673f034feeef2              0     0     0     1   
...                                ...            ...   ...   ...   ...   
9907  ffbfa0383c34dc513c95560d6e1fdb57              0     0     0     1   
9908  ffcc8532d76436fc79e50eb2e5238e45              0     1     1     1   
9909  ffdf2e8673a1da6fb80342fa3b119a20              0     1     1     1   
9910  fff19e2ce11718548fa1c5d039a5192a              0     1     1     1   
9911  fff8e47c766799c9e12f3cb3d66ad228              

In [3]:
print('shuffling')
# Shuffling data and separating into train, test and dev sets
shuffle = np.random.permutation(np.arange(X_train.shape[0]))
X_train, labels_raw, labels_bin, tags = X_train[shuffle], labels_raw[shuffle], labels_bin[shuffle], tags[shuffle]
print('shuffled')
print('data shape: ', X_train.shape)
print('pawpularity label shape:', labels_raw.shape)
print('bin label shape:', labels_bin.shape)
print('tags  shape:', tags.shape)

# Set some variables to hold test, dev, and training data.
train_data, train_labels_raw, train_labels_bin, train_tags = X_train[:7921], labels_raw[:7921], labels_bin[:7921], tags[:7921]
test_data, test_labels_raw, test_labels_bin, test_tags = X_train[8916:], labels_raw[8916:], labels_bin[8916:], tags[8916:]
dev_data, dev_labels_raw, dev_labels_bin, dev_tags = X_train[7920:8916], labels_raw[7920:8916], labels_bin[7920:8916], tags[7920:8916]

print('test_labels_raw', test_labels_raw.shape)
print('train_labels_raw', train_labels_raw.shape)
print('dev_labels_raw', dev_labels_raw.shape)

# raw labels (ie original pawpularity score)
test_labels_category = to_categorical(test_labels_raw)
train_labels_category = to_categorical(train_labels_raw)
dev_labels_category = to_categorical(dev_labels_raw)

print('test_labels_category', test_labels_category.shape)
print('train_labels_category', train_labels_category.shape)
print('dev_labels_category', dev_labels_category.shape)

# bucketed labels 
test_labels_bins_category = to_categorical(test_labels_bin)
train_labels_bins_category = to_categorical(train_labels_bin)
dev_labels_bins_category = to_categorical(dev_labels_bin)

# blurred image data (optional use)
blurred_train = gaussian_filter(train_data, sigma=1)
blurred_test = gaussian_filter(test_data, sigma = 1)

shuffling
shuffled
data shape:  (9912, 50, 50, 3)
pawpularity label shape: (9912,)
bin label shape: (9912,)
tags  shape: (9912, 11)
test_labels_raw (996,)
train_labels_raw (7921,)
dev_labels_raw (996,)
test_labels_category (996, 101)
train_labels_category (7921, 101)
dev_labels_category (996, 101)


# First CNN Model 

In [4]:
shape = (train_data.shape[1],train_data.shape[2], train_data.shape[3])
possible_outcomes = 101
# first model attempt (using raw pawpularity scores)
# switch out activation to 'sigmoid'
# try different pixel amounts (100x100) (20x20) etc
#try binary crosstentropy vs categorical 
#adjust epochs


model = Sequential()#add model layers
model.add(Conv2D(32, kernel_size=(5, 5),
                     activation='relu',
                     input_shape=shape))

model.add(MaxPooling2D(pool_size=(2, 2)))
# add second convolutional layer with 20 filters
model.add(Conv2D(64, (5, 5), activation='relu'))
    
# add 2D pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))
    
# flatten data
model.add(Flatten())
    
# add a dense all-to-all relu layer
model.add(Dense(1024, activation='relu'))
    
# apply dropout with rate 0.5
model.add(Dropout(0.5))
    
# soft-max layer
model.add(Dense(possible_outcomes, activation='softmax'))
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#train the model

print('-'*10, 'CNN Model 1 with all Paw scores', '-'*10)
print("train_data",train_data.shape )
print("train_lables_cat",train_labels_category.shape )
print("test_data",test_data.shape )
print("test_lables_cat",test_labels_category.shape )

model.fit(train_data,train_labels_category, validation_data=(test_data, test_labels_category), epochs=3)
# evaluate the model
score = model.evaluate(test_data, test_labels_category, verbose=1)
# print performance
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

---------- CNN Model 1 with all Paw scores ----------
train_data (7921, 50, 50, 3)
train_lables_cat (7921, 101)
test_data (996, 50, 50, 3)
test_lables_cat (996, 101)
Epoch 1/3
248/248 [==============================] - 29s 113ms/step - loss: 4.2656 - accuracy: 0.0276 - val_loss: 4.2411 - val_accuracy: 0.0361
Epoch 2/3
248/248 [==============================] - 30s 121ms/step - loss: 4.2260 - accuracy: 0.0288 - val_loss: 4.2148 - val_accuracy: 0.0311
Epoch 3/3
32/32 [==============================] - 0s 15ms/step - loss: 4.2060 - accuracy: 0.0371

Test loss: 4.206021785736084
Test accuracy: 0.03714859485626221


# Same Model but using 10 category buckets for scores

In [5]:
shape = (train_data.shape[1],train_data.shape[2], train_data.shape[3])
possible_outcomes = 10
# first model attempt (using raw pawpularity scores)
# switch out activation to 'sigmoid'
# try different pixel amounts (100x100) (20x20) etc
#try binary crosstentropy vs categorical 
#adjust epochs


model = Sequential()#add model layers
model.add(Conv2D(32, kernel_size=(5, 5),
                     activation='relu',
                     input_shape=shape))

model.add(MaxPooling2D(pool_size=(2, 2)))
# add second convolutional layer with 20 filters
model.add(Conv2D(64, (5, 5), activation='relu'))
    
# add 2D pooling layer
model.add(MaxPooling2D(pool_size=(2, 2)))
    
# flatten data
model.add(Flatten())
    
# add a dense all-to-all relu layer
model.add(Dense(1024, activation='relu'))
    
# apply dropout with rate 0.5
model.add(Dropout(0.5))
    
# soft-max layer
model.add(Dense(possible_outcomes, activation='softmax'))
#compile model using accuracy to measure model performance
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
#train the model

print('-'*10, 'CNN Model 1 with all Paw scores', '-'*10)
model.fit(train_data,train_labels_bins_category, validation_data=(test_data, test_labels_bins_category), epochs=3)
# evaluate the model
score = model.evaluate(test_data, test_labels_bins_category, verbose=1)
# print performance
print()
print('Test loss:', score[0])
print('Test accuracy:', score[1])

---------- CNN Model 1 with all Paw scores ----------
Epoch 1/3
248/248 [==============================] - 26s 104ms/step - loss: 2.0004 - accuracy: 0.2613 - val_loss: 1.9922 - val_accuracy: 0.2922
Epoch 2/3
248/248 [==============================] - 23s 93ms/step - loss: 1.9806 - accuracy: 0.2729 - val_loss: 1.9790 - val_accuracy: 0.2922
Epoch 3/3
32/32 [==============================] - 0s 13ms/step - loss: 1.9691 - accuracy: 0.2922

Test loss: 1.969141960144043
Test accuracy: 0.29216867685317993


# 2Parameter Second CNN Model - Using raw paw scores

In [11]:
# model from kaggle page 
# https://www.kaggle.com/alexteboul/tutorial-part-3-cnn-image-modeling-1

print(train_tags.shape)
print(train_data.shape)
print(train_labels_category.shape)
print(type(train_tags))
print(type(train_data))
print(type(train_labels_category))
# this accuracy seems ot be higher we could mess around with different filters/padding etc
shape = (train_data.shape[1],train_data.shape[2], train_data.shape[3])
inputs = tf.keras.Input(shape=shape)
tags_input = tf.keras.Input(shape=(train_tags.shape[1],))
#start off with x just being those inputs
x = inputs

x = tf.keras.layers.Conv2D(filters = 16, kernel_size = (7,7), strides = (2,2), padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding='same', activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), strides = (2,2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)


x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), strides = (2,2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation = "relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.Model(inputs=inputs, outputs=tf.keras.layers.Dense(1)(x))

y = tags_input
y = tf.keras.layers.Dense(11, activation="relu")(y)
y = tf.keras.layers.Dense(5, activation="relu")(y)
y = tf.keras.Model(inputs=tags_input, outputs=y) 

# combine the output of the two branches
combined = tf.keras.layers.concatenate([x.output, y.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
z = Dense(2, activation="relu")(combined)
z = Dense(1, activation="linear")(z)
# our model will accept the inputs of the two branches and
# then output a single value
model2 = tf.keras.Model(inputs=[x.input, y.input], outputs=z)

#model = tf.keras.Model(inputs = [inputs], outputs = output)
#model2 = tf.keras.Model(inputs = [inputs, tags_input], outputs = output)

model2.compile(
    loss = 'mse', 
    optimizer = 'Adam', 
    metrics = [tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])

x1=tf.convert_to_tensor(train_data)
x2=tf.convert_to_tensor(train_tags)
sx1=tf.convert_to_tensor(test_data)
sx2=tf.convert_to_tensor(test_tags)

model2.fit(x=[x1,x2], y=tf.convert_to_tensor(train_labels_category), batch_size=32, epochs=3, validation_split=0.2)
score = model2.evaluate(x=[sx1,sx2], y=test_labels_category, verbose=1)
print('Test accuracy:', score[1])



(7921, 11)
(7921, 50, 50, 3)
(7921, 101)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
Epoch 1/3
198/198 [==============================] - 12s 53ms/step - loss: 0.2281 - rmse: 0.1077 - mae: 0.0186 - mape: 8810581.0000 - val_loss: 0.2042 - val_rmse: 0.1011 - val_mae: 0.0238 - val_mape: 14001861.0000
Epoch 2/3
198/198 [==============================] - 10s 52ms/step - loss: 0.1850 - rmse: 0.0990 - mae: 0.0196 - mape: 9802909.0000 - val_loss: 0.1672 - val_rmse: 0.0990 - val_mae: 0.0196 - val_mape: 9802953.0000
Epoch 3/3
32/32 [==============================] - 1s 15ms/step - loss: 0.1370 - rmse: 0.0990 - mae: 0.0196 - mape: 9802959.0000
Test accuracy: 0.09900990128517151


# 2Parameter Second CNN Model - Using bucket paw scores

In [12]:
# model from kaggle page 
# https://www.kaggle.com/alexteboul/tutorial-part-3-cnn-image-modeling-1

# this accuracy seems ot be higher we could mess around with different filters/padding etc
shape = (train_data.shape[1],train_data.shape[2], train_data.shape[3])
inputs = tf.keras.Input(shape=shape)
tags_input = tf.keras.Input(shape=(train_tags.shape[1],))

#start off with x just being those inputs
x = inputs

x = tf.keras.layers.Conv2D(filters = 16, kernel_size = (7,7), strides = (2,2), padding='valid', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), padding='same', activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters = 32, kernel_size = (3,3), strides = (2,2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)


x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3), strides = (2,2), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0005), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3), padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPooling2D(pool_size=(2, 2))(x)
x = tf.keras.layers.Conv2D(filters = 128, kernel_size = (3,3),padding='same', kernel_initializer='he_normal', kernel_regularizer=l2(0.0002), activation = 'relu')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.25)(x)

x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(512, activation = "relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)

y = tags_input
y = tf.keras.layers.Dense(11, activation="relu")(y)
y = tf.keras.layers.Dense(5, activation="relu")(y)
y = tf.keras.Model(inputs=tags_input, outputs=y) 

# combine the output of the two branches
combined = tf.keras.layers.concatenate([x.output, y.output])
# apply a FC layer and then a regression prediction on the
# combined outputs
z = Dense(2, activation="relu")(combined)
z = Dense(1, activation="linear")(z)
# our model will accept the inputs of the two branches and
# then output a single value
model2 = tf.keras.Model(inputs=[x.input, y.input], outputs=z)

model2.compile(
    loss = 'mse', 
    optimizer = 'Adam', 
    metrics = [tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])

x1=tf.convert_to_tensor(train_data)
x2=tf.convert_to_tensor(train_tags)
sx1=tf.convert_to_tensor(test_data)
sx2=tf.convert_to_tensor(test_tags)

model2.fit(x=[x1,x2], y=tf.convert_to_tensor(train_labels_bins_category), batch_size=32, epochs=3, validation_split=0.2)
score = model2.evaluate(x=[sx1,sx2], y=test_labels_bins_category, verbose=1)
print('Test accuracy:', score[1])

#model = tf.keras.Model(inputs = [inputs, tags_input], outputs = output)

#model.compile(
#    loss = 'mse', 
#    optimizer = 'Adam', 
#    metrics = [tf.keras.metrics.RootMeanSquaredError(name="rmse"), "mae", "mape"])

#data_augmentation = ImageDataGenerator(
#    rotation_range = 15, 
#    zoom_range = 0.15,
#    width_shift_range = 0.2, 
#    height_shift_range = 0.2, 
#    shear_range = 0.1,
#    horizontal_flip = True, 
#    fill_mode = "nearest")
#history = model.fit(
#    data_augmentation.flow(train_data,train_labels_bins_category,batch_size=32),
#    validation_data = (test_data, test_labels_bins_category),
#    steps_per_epoch = len(train_data) // 32,
#    epochs = 3
#)
#score = model.evaluate(test_data, test_labels_bins_category, verbose=1)
#print('Test accuracy:', score[1])




AttributeError: 'KerasTensor' object has no attribute 'output'